# Import Stuff

In [1]:
import numpy as np
import scipy as sp
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import networkx as nx

# Default Code, Don't Edit This

In [2]:
from ema_workbench import (Model, CategoricalParameter,
                           ScalarOutcome, IntegerParameter, RealParameter)
from dike_model_function import DikeNetwork  # @UnresolvedImport

def sum_over(*args):
    return sum(args)

In [3]:
from ema_workbench import (Model, MultiprocessingEvaluator, Policy, Scenario)

from ema_workbench.em_framework.evaluators import perform_experiments
from ema_workbench.em_framework.samplers import sample_uncertainties
from ema_workbench.util import ema_logging
import time
from problem_formulation import get_model_for_problem_formulation


ema_logging.log_to_stderr(ema_logging.INFO)

#choose problem formulation number, between 0-5
#each problem formulation has its own list of outcomes
dike_model, planning_steps = get_model_for_problem_formulation(3)

In [4]:
#enlisting uncertainties, their types (RealParameter/IntegerParameter/CategoricalParameter), lower boundary, and upper boundary
#for unc in dike_model.uncertainties:
    #print(repr(unc))
    
uncertainties = dike_model.uncertainties

import copy
uncertainties = copy.deepcopy(dike_model.uncertainties)

In [5]:
#enlisting policy levers, their types (RealParameter/IntegerParameter), lower boundary, and upper boundary
#for policy in dike_model.levers:
    #print(repr(policy))
    
levers = dike_model.levers 

import copy
levers = copy.deepcopy(dike_model.levers)

In [6]:
#enlisting outcomes
#for outcome in dike_model.outcomes:
    #print(repr(outcome))

# Creating the Policies
### Edit this stuff

In [7]:
#defining specific policies
#for example, policy 1 is about extra protection in upper boundary
#policy 2 is about extra protection in lower boundary
#policy 3 is extra protection in random locations
from ema_workbench import Policy
#testing Secnario 0 - Baseline
policies = [Policy('policy 0', **{'0_RfR 0':0,
                                  '0_RfR 1':0,
                                  '0_RfR 2':0,
                                  'A.1_DikeIncrease 0':[7-10],
                                  'A.1_DikeIncrease 1':[7-10],
                                  'A.1_DikeIncrease 2':[7-10],
                                  '1_RfR 0':0,
                                  '1_RfR 1':0,
                                  '1_RfR 2':0,
                                  # 'A.2_DikeIncrease 0':[7-10],
                                  # 'A.2_DikeIncrease 1':[7-10],
                                  # 'A.2_DikeIncrease 2':[7-10],
                                  # '2_RfR 0':0,
                                  # '2_RfR 1':0,
                                  # '2_RfR 2':0,
                                  # 'A.3_DikeIncrease 0':[7-10],
                                  # 'A.3_DikeIncrease 1':[7-10],
                                  # 'A.3_DikeIncrease 2':[7-10],
                                  # '3_RfR 0':0,
                                  # '3_RfR 1':0,
                                  # '3_RfR 2':0,
                                  # 'A.4_DikeIncrease 0':[7-10],
                                  # 'A.4_DikeIncrease 1':[7-10],
                                  # 'A.4_DikeIncrease 2':[7-10],
                                  # '4_RfR 0':0,
                                  # '4_RfR 1':0,
                                  # '4_RfR 2':0,
                                  # 'A.5_DikeIncrease 0':[7-10],
                                  # 'A.5_DikeIncrease 1':[7-10],
                                  # 'A.5_DikeIncrease 2':[7-10],
                                  }),]

# Running the Model
### You can edit the number of scenarios to help adjust runtime, more is kind of better

In [ ]:
#pass the policies list to EMA workbench experiment runs
n_scenarios = 1
with MultiprocessingEvaluator(dike_model) as evaluator:
    results = evaluator.perform_experiments(n_scenarios,
                                            policies)

[MainProcess/INFO] pool started with 12 workers
[MainProcess/INFO] performing 1 scenarios * 1 policies * 1 model(s) = 1 experiments
  0%|                                                    | 0/1 [00:00<?, ?it/s]

In [ ]:
#print(results)

In [ ]:
experiments, outcomes = results

### Edit the column for outcome to see different results

In [ ]:
from ema_workbench.analysis import prim
x = experiments
y = outcomes["A.2 Total Costs"] < 2.5e+08
prim_alg = prim.Prim(x, y, threshold=0.8)
box1 = prim_alg.find_box()

In [ ]:
box1.show_tradeoff()
plt.show()

### Edit the column for outcome to see different results

In [ ]:
from ema_workbench.analysis import dimensional_stacking

x = experiments
y = outcomes["A.2 Total Costs"] < 1
dimensional_stacking.create_pivot_plot(x, y, 2, nbins=3)
plt.show()


### change which parts of the experiments table it outputs to see the figure better

In [ ]:
from ema_workbench.analysis import feature_scoring

x = experiments.loc[:,['A.1_pfail', 'A.2_pfail', 'A.3_pfail', 'A.4_pfail', 'A.5_pfail']]
y = outcomes

fs = feature_scoring.get_feature_scores_all(x, y)
sns.heatmap(fs, cmap="viridis", annot=True, )
plt.figure(figsize = (100,100))
plt.show()